In [1]:
from helpers import *

import sys

import bz2
import json

import pickle

import numpy as np
# import scipy

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score, cross_validate

%load_ext autoreload
%autoreload 2

## Finding the optimal amount of preprocessing

## Full data model training

In [2]:
path = fixpath(QUOTES_LABELED_CLEANED)
# path = fixpath(QUOTES_2020_LABELED_CLEANED)
df_raw = pd.read_json(path, orient='records', lines=True)
df_raw

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_clean
0,2015-08-31-000271,... a great day for veterans here in Littleton...,jeanne shaheen,2015-08-31 02:10:00,2,Q270316,D,S001181,great day veteran littleton across north count...
1,2015-02-03-074650,The safety and security of our nation is at ri...,jeanne shaheen,2015-02-03 20:02:24,2,Q270316,D,S001181,safety security nation risk hold funding homel...
2,2015-05-10-027625,It's been my experience that the faith communi...,jeanne shaheen,2015-05-10 22:48:46,1,Q270316,D,S001181,experience faith community willing partner gov...
3,2015-09-01-113454,we need to bring the parents in so they can se...,jeanne shaheen,2015-09-01 02:12:00,2,Q270316,D,S001181,need bring parent see manufacturing worked sho...
4,2015-10-25-000242,"' It is not now, nor has it ever been, the gol...",bernie sanders,2015-10-25 14:12:35,1,Q359442,D,S000033,ever gold standard trade agreement sander said...
...,...,...,...,...,...,...,...,...,...
6396366,2020-02-10-100845,We're just worried about making sure we keep t...,sherrie sprenger,2020-02-10 00:00:00,1,Q7495360,R,None,worried making sure keep balance expression ri...
6396367,2020-03-16-079753,We've all embraced strict proper hygiene proce...,robert abrams,2020-03-16 12:00:00,2,Q2156314,D,None,embraced strict proper hygiene procedure heard...
6396368,2020-01-13-091997,What's important is that we keep moving forward.,laurie jinkins,2020-01-13 19:51:15,1,Q6501617,D,None,important keep moving forward
6396369,2020-02-20-093793,"When they want a bill, they bring the bill to ...",j.t. wilcox,2020-02-20 22:12:45,1,Q6104393,R,None,want bill bring bill floor


In [3]:
df = df_raw.copy()
df.drop(['numOccurrences','US_congress_bio_ID'], axis=1, inplace=True)
df

,quoteID,quotation,speaker,date,id,party_label,quotation_clean
0,2015-08-31-000271,... a great day for veterans here in Littleton...,jeanne shaheen,2015-08-31 02:10:00,Q270316,D,great day veteran littleton across north count...
1,2015-02-03-074650,The safety and security of our nation is at ri...,jeanne shaheen,2015-02-03 20:02:24,Q270316,D,safety security nation risk hold funding homel...
2,2015-05-10-027625,It's been my experience that the faith communi...,jeanne shaheen,2015-05-10 22:48:46,Q270316,D,experience faith community willing partner gov...
3,2015-09-01-113454,we need to bring the parents in so they can se...,jeanne shaheen,2015-09-01 02:12:00,Q270316,D,need bring parent see manufacturing worked sho...
4,2015-10-25-000242,"' It is not now, nor has it ever been, the gol...",bernie sanders,2015-10-25 14:12:35,Q359442,D,ever gold standard trade agreement sander said...
...,...,...,...,...,...,...,...
6396366,2020-02-10-100845,We're just worried about making sure we keep t...,sherrie sprenger,2020-02-10 00:00:00,Q7495360,R,worried making sure keep balance expression ri...
6396367,2020-03-16-079753,We've all embraced strict proper hygiene proce...,robert abrams,2020-03-16 12:00:00,Q2156314,D,embraced strict proper hygiene procedure heard...
6396368,2020-01-13-091997,What's important is that we keep moving forward.,laurie jinkins,2020-01-13 19:51:15,Q6501617,D,important keep moving forward
6396369,2020-02-20-093793,"When they want a bill, they bring the bill to ...",j.t. wilcox,2020-02-20 22:12:45,Q6104393,R,want bill bring bill floor


In [4]:
df['party_label'].value_counts()

D    3224525
R    3171846
Name: party_label, dtype: int64

### Filtering politcally insignificant quotes 

In [5]:
def filter_relevant_quotes_basic(df):
    lengths_of_df = df['quotation_clean'].apply(lambda x: len(x))

    # df = df[np.logical_and(lengths_of_df > lengths_of_df.quantile(0.3), lengths_of_df < lengths_of_df.quantile(0.9))]
    df = df[lengths_of_df > lengths_of_df.quantile(0.1)]

    return df

In [6]:
df_filt = filter_relevant_quotes_basic(df)
df_filt

,quoteID,quotation,speaker,date,id,party_label,quotation_clean
0,2015-08-31-000271,... a great day for veterans here in Littleton...,jeanne shaheen,2015-08-31 02:10:00,Q270316,D,great day veteran littleton across north count...
1,2015-02-03-074650,The safety and security of our nation is at ri...,jeanne shaheen,2015-02-03 20:02:24,Q270316,D,safety security nation risk hold funding homel...
2,2015-05-10-027625,It's been my experience that the faith communi...,jeanne shaheen,2015-05-10 22:48:46,Q270316,D,experience faith community willing partner gov...
3,2015-09-01-113454,we need to bring the parents in so they can se...,jeanne shaheen,2015-09-01 02:12:00,Q270316,D,need bring parent see manufacturing worked sho...
4,2015-10-25-000242,"' It is not now, nor has it ever been, the gol...",bernie sanders,2015-10-25 14:12:35,Q359442,D,ever gold standard trade agreement sander said...
...,...,...,...,...,...,...,...
6396366,2020-02-10-100845,We're just worried about making sure we keep t...,sherrie sprenger,2020-02-10 00:00:00,Q7495360,R,worried making sure keep balance expression ri...
6396367,2020-03-16-079753,We've all embraced strict proper hygiene proce...,robert abrams,2020-03-16 12:00:00,Q2156314,D,embraced strict proper hygiene procedure heard...
6396368,2020-01-13-091997,What's important is that we keep moving forward.,laurie jinkins,2020-01-13 19:51:15,Q6501617,D,important keep moving forward
6396369,2020-02-20-093793,"When they want a bill, they bring the bill to ...",j.t. wilcox,2020-02-20 22:12:45,Q6104393,R,want bill bring bill floor


In [7]:
df_filt['party_label'].value_counts()

D    2911651
R    2844867
Name: party_label, dtype: int64

In [8]:
df_filt['quotation_clean'].apply(lambda x: len(x)).min()

23

### Balancing data

In [9]:

df_filt = downsample(df_filt, 'party_label')

In [10]:
df_filt['party_label'].value_counts()

D    2844867
R    2844867
Name: party_label, dtype: int64

In [11]:
df_filt

,quoteID,quotation,speaker,date,id,party_label,quotation_clean
0,2019-09-25-034841,I humbly ask the Florida Senate to approve my ...,scott israel,2019-09-25 18:31:52,Q50591793,D,humbly ask florida senate approve reinstatemen...
1,2018-10-19-034441,"I don't care who you vote for, I just want you...",sonia sotomayor,2018-10-19 22:17:37,Q11107,D,care vote want voice change
2,2018-05-01-037575,"I am really happy in business community,",wesley clark,2018-05-01 17:19:20,Q159478,D,really happy business community
3,2016-05-02-059378,nobody's falling through [ cracks because of ]...,rahm emanuel,2016-05-02 22:28:02,Q298443,D,nobody falling crack siloed wall approach
4,2019-07-03-048691,Marianne Williamson has already reached legend...,marianne williamson,2019-07-03 10:29:00,Q3131983,D,marianne williamson already reached legendary ...
...,...,...,...,...,...,...,...
5689729,2018-05-25-114419,U.S. officials were further frustrated last we...,mike pompeo,2018-05-25 06:12:40,Q473239,R,u official frustrated last week senior north k...
5689730,2019-11-12-036716,I think that those veterans who are improperly...,tom tiffany,2019-11-12 00:23:08,Q7817838,R,think veteran improperly charged refunded money
5689731,2019-03-18-060255,Our men and women in uniform oftentimes risk t...,john mcguire,2019-03-18 17:33:58,Q47545631,R,men woman uniform oftentimes risk life even sa...
5689732,2017-08-31-091904,"NBN has a responsibility to our shareholders, ...",bill morrow,2017-08-31 06:09:39,Q4910265,R,nbn responsibility shareholder virtually citiz...


## Model Training

In [12]:
# df_mini = df_filt.sample(1000)
df_mini = df_filt.sample(frac=1)

In [13]:
X = df_mini['quotation_clean'].values

y = df_mini['party_label'].values
y = np.array([0.0 if label=='R' else 1.0 for label in y])

In [14]:
split_frac = 0.9

bound = round(len(X)*split_frac)

X_train = X[:bound]
y_train = y[:bound]

X_test = X[bound:]
y_test = y[bound:]

In [15]:
X_train.shape

(5120761,)

In [16]:
def vectorize_with_file(X, vectorizer):
    path_temp = TEMP_FILE

    with open(path_temp, 'w') as d_file:
        d_file.writelines(X + '\n')

    with open(path_temp, 'r') as s_file:
        X_vect=vectorizer.fit_transform(s_file)

    return X_vect

In [26]:
# Vectorize by writing to file
vectorizer=TfidfVectorizer(ngram_range=(1,3))
X_vect = vectorize_with_file(X_train, vectorizer)

In [27]:
with open(TEMP_FOLDER + 'quotes_vectorized_ngram=(1,3).pkl', 'wb') as file:
    pickle.dump(X_vect, file)

In [28]:
# Vectorize in memory
# vectorizer2=TfidfVectorizer()
# X_vect2 = vectorizer2.fit_transform(X_train)

In [29]:
X_test_vect = vectorizer.transform(X_test)

In [30]:
X_vect.shape

(5120761, 44225945)

In [33]:
with open(TEMP_FOLDER + 'vectorizer_ngram=(1,3)-multinomialnb.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [ ]:
with open(TEMP_FOLDER + 'vectorizer_ngram=(1,3)-multinomialnb.pkl', 'rb') as file:
    vectorizer_loaded = pickle.load(file)

In [31]:
clf = MultinomialNB()

clf.fit(X_vect, y_train)
clf.score(X_test_vect, y_test)


0.7230307940798597

In [32]:
with open(TEMP_FOLDER + 'model-tfidf_ngram=(1,3)-multinomialnb.pkl', 'wb') as file:
    pickle.dump(clf, file)

In [24]:
with open(TEMP_FOLDER + 'model-tfidf_ngram=(1,3)-multinomialnb.pkl', 'rb') as file:
    clf_loaded = pickle.load(file)

In [25]:
clf_loaded.score(X_test_vect, y_test)

0.7012336262001888

In [ ]:
TfidfVectorizer(TEMP_FILE, )

In [28]:
pipeline_simple = Pipeline([
        ('vect', TfidfVectorizer(ngram_range=(1,1), max_features=100_000)),  
        ('clf', MultinomialNB())
    ],
    verbose=True
    )

In [29]:
# Run cross validation

scoring=['accuracy', 'precision', 'recall', 'f1']
res = cross_validate(pipeline_simple, X, y, scoring=scoring, cv=2, n_jobs=2)
print_cross_validate_results(res)

[Pipeline] .............. (step 1 of 2) Processing vect, total=   2.1s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.0s
[Pipeline] .............. (step 1 of 2) Processing vect, total=   2.1s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.0s
	fit_time             - 	avg: 2.138	std: 0.021
	score_time           - 	avg: 2.696	std: 0.015
	test_accuracy        - 	avg: 0.662	std: 0.001
	test_precision       - 	avg: 0.663	std: 0.001
	test_recall          - 	avg: 0.661	std: 0.001
	test_f1              - 	avg: 0.662	std: 0.000


In [30]:
split_frac = 0.9

bound = round(len(X)*split_frac)

X_train = X[:bound]
y_train = y[:bound]

X_test = X[bound:]
y_test = y[bound:]

In [31]:
X_train.shape, X_test.shape

((210749,), (23417,))

In [32]:
pipeline_simple.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 2) Processing vect, total=   3.5s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.1s


Pipeline(steps=[('vect', TfidfVectorizer(max_features=100000)),
                ('clf', MultinomialNB())],
         verbose=True)

In [33]:
pipeline_simple.score(X_test, y_test)

0.6719050262629713

In [35]:
pipeline_simple.predict(X[0:10])

array([1., 1., 1., 0., 0., 0., 0., 1., 1., 1.])

In [39]:
cachedir = mkdtemp()

In [50]:
from tempfile import mkdtemp
from shutil import rmtree
from joblib import Memory

# Create a temporary folder to store the transformers of the pipeline

memory = Memory(location=cachedir, verbose=10)
# cached_pipe = Pipeline([('reduce_dim', PCA()),
#                         ('classify', LinearSVC())],
#                        memory=memory)

pipeline_adv = Pipeline([
        ('vect', TfidfVectorizer(ngram_range=(1,3), max_features=1_000_000)),  
        ('clf', MultinomialNB())
    ],
    memory=memory,
    verbose=True
    )

In [47]:
scoring=['accuracy', 'precision', 'recall', 'f1']
res = cross_validate(pipeline_simple, X, y, scoring=scoring, cv=3, n_jobs=1)
print_cross_validate_results(res)

[Pipeline] .............. (step 1 of 2) Processing vect, total=   0.1s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.0s
[Pipeline] .............. (step 1 of 2) Processing vect, total=   0.1s
[Pipeline] .............. (step 1 of 2) Processing vect, total=   0.1s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.0s
	fit_time             - 	avg: 0.064	std: 0.002
	score_time           - 	avg: 0.019	std: 0.001
	test_accuracy        - 	avg: 0.519	std: 0.009
	test_precision       - 	avg: 0.520	std: 0.008
	test_recall          - 	avg: 0.571	std: 0.041
	test_f1              - 	avg: 0.543	std: 0.022


In [54]:
pipeline_adv.fit(X_train, y_train)

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfVectorizer(max_features=1000000, ngram_range=(1, 3)), array(['service people endured every single day behalf never ever forgotten',
       ..., 'prepared work issue solution negotiation give take'],
      dtype=object), 
array([1., ..., 0.]), None, message_clsname='Pipeline', message='(step 1 of 2) Processing vect')
[Pipeline] .............. (step 1 of 2) Processing vect, total=14.7min


/Users/mroust/anaconda3/envs/ada/lib/python3.8/site-packages/sklearn/pipeline.py:348: UserWarning: Persisting input arguments took 685.58s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(


____________________________________________fit_transform_one - 1432.9s, 23.9min
[Pipeline] ............... (step 2 of 2) Processing clf, total=  10.3s


Pipeline(memory=Memory(location=/var/folders/nf/186rd3kx35v_x13p5402wpsr0000gn/T/tmp494ki5dy/joblib),
         steps=[('vect',
                 TfidfVectorizer(max_features=1000000, ngram_range=(1, 3))),
                ('clf', MultinomialNB())],
         verbose=True)

In [66]:
pipeline_adv.score(X_test, y_test)

0.549641195627912

In [58]:
with open(MODEL_MULTINOMIALNB, 'wb') as file:
    pickle.dump(pipeline_adv, file)

In [59]:
MODEL_MULTINOMIALNB



'models/model_multinomialnb.pkl'

In [45]:
# Delete the temporary cache before exiting
rmtree(cachedir)